In [2]:
%matplotlib inline
import json

In [10]:
import json
with open('../test/two-sigma-connect-rental-listing-inquiries/train.json') as train:
    json_data = json.load(train)
df = pd.DataFrame(json_data)

## EDA
- 피쳐 데이터 타입 살펴보기
- missing data 확인
- target variable 분포 확인
- price 분포 확인
- longitude/latitude 분포 확인
- bathroom, bedroom 분포 확인
- numeric feature 공선성 확인

In [ ]:
df.shape

In [ ]:
# 피쳐 데이터 타입 살펴보기
df_dtype = df.dtypes.reset_index()
df_dtype.columns = ['Count', 'Column Type']
df_dtype.groupby('Column Type').aggregate('count').reset_index()

In [ ]:
# missing data 없음
df.isnull().sum()

In [ ]:
# Target variable exploration
sns.countplot(df.interest_level, order=['low', 'medium', 'high']);
plt.xlabel('Interest Level');
plt.ylabel('Number of occurrences');

In [ ]:
# price histogram
def price_histogram():
    fig,ax = plt.subplots(1,2, figsize=(20,8))
    ax1,ax2 = ax.flatten()

    df.price.plot.hist(bins=20, color='Purple', edgecolor='black', ax=ax1);
    ax1.set_title('Price Histograms', fontsize=17)
    ax1.set_ylabel('')
    ax1.set_xlabel('')

    log_price = np.log(df.price)
    sns.distplot(log_price,  bins=30, color='tomato', kde=True, ax=ax2)
    ax2.set_title("Price Distribution Histogram", fontsize=17)
    ax2.set_ylabel("")
    ax2.set_xlabel('')
    plt.subplots_adjust(wspace=0.2);

price_histogram()

In [ ]:
# Average Price per Interest Level
sns.barplot(x="interest_level", y="price", order=['low', 'medium', 'high'], data=df)
plt.xlabel('Interest Level')
plt.ylabel('Price')
plt.show()

In [ ]:
# longtitude/ latitude 분포 확인
sns.pairplot(df[['longitude', 'latitude', 'interest_level']], hue='interest_level')
plt.ylabel('latitude')
plt.xlabel('longitude')
plt.show()

In [ ]:
# Bedrooms graphs
fig = plt.figure(figsize=(12,12))
# Number of occurrences
sns.countplot(df.bedrooms, ax = plt.subplot(221));
plt.xlabel('Number of Bedrooms');
plt.ylabel('Number of occurrences');
# Average number of Bathrooms per Interest Level
sns.barplot(x='interest_level', y='bedrooms', data=df, order=['low', 'medium', 'high'],
            ax = plt.subplot(222));
plt.xlabel('Interest Level');
plt.ylabel('Average Number of Bathrooms');

In [ ]:
# Bathrooms graphs
fig = plt.figure(figsize=(12,12))
# Number of occurrences
sns.countplot(df.bathrooms, ax = plt.subplot(221));
plt.xlabel('Number of Bathrooms');
plt.ylabel('Number of occurrences');
# Average number of Bathrooms per Interest Level
sns.barplot(x='interest_level', y='bathrooms', data=df, order=['low', 'medium', 'high'],
            ax = plt.subplot(222));
plt.xlabel('Interest Level');
plt.ylabel('Average Number of Bathrooms');

In [ ]:
# numeric feature 공선성 확인
df_numeric = df.select_dtypes(include=['int64', 'float64'])
plt.figure(figsize=(20,20))
sns.heatmap(data = df_numeric.corr(), annot=True, fmt = '.2f', linewidths=.5, cmap='Blues')

## feature engineering
- 'created' 피쳐를 게시된 년도, 월, 날짜, 시간, 주말/평일 구분 column 추가
- price per bad column 추가
- price outlier 확인 및 상/하위 1% outlier 제거
- log price column 추가
- number of photos column 추가
- description length column 추가
- sutdio column(bedroom = 0)추가
- NY 벗어나는 위도경도 outlier 제거
- features column에서 상위 10개 새로운 column으로 생성
#### 고민해보기
- neighborhood column 
- building_id, manager_id 

In [11]:
# 게시된 년도, 월, 날짜, 시간 column 생성
df.created = pd.to_datetime(df.created, format='%Y-%m-%d %H:%M:%S')
df['month'] = df.created.dt.month
df['day_of_week'] = df.created.dt.weekday_name
df['hour'] = df.created.dt.hour
# 'created' column drop
df.drop('created', axis=1, inplace=True)

In [12]:
# 주말(1)/평일(0)  column 생성
df['weekend'] = df.apply(lambda row: 1 if row['day_of_week'] in ['Saturday', 'Sunday', 'Friday'] else 0, axis=1)

In [ ]:
# Iterest per month
fig = plt.figure(figsize=(12,6))
ax = sns.countplot(x="month", hue="interest_level", hue_order=['low', 'medium', 'high'],
                   data=df);
plt.xlabel('Month');
plt.ylabel('Number of occurrences')

# Adding percents over bars
height = [p.get_height() for p in ax.patches]
ncol = int(len(height)/3)
total = [height[i] + height[i + ncol] + height[i + 2*ncol] for i in range(ncol)] * 3
for i, p in enumerate(ax.patches):    
    ax.text(p.get_x()+p.get_width()/2,
            height[i] + 50,
            '{:1.0%}'.format(height[i]/total[i]),
            ha="center") 

In [ ]:
# # Iterest per Day of Week
# fig = plt.figure(figsize=(12,6))
# ax = sns.countplot(x="day_of_week", hue="interest_level",
#                    hue_order=['low', 'medium', 'high'], data=df,
#                    order=['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']);
# plt.xlabel('Day of Week');
# plt.ylabel('Number of occurrences');

# # Adding percents over bars
# height = [p.get_height() for p in ax.patches]
# ncol = int(len(height)/3)
# total = [height[i] + height[i + ncol] + height[i + 2*ncol] for i in range(ncol)] * 3
# for i, p in enumerate(ax.patches):    
#     ax.text(p.get_x()+p.get_width()/2,
#             height[i] + 50,
#             '{:1.0%}'.format(height[i]/total[i]),
#             ha="center") 

In [ ]:
# Day of Week per Iterest
fig = plt.figure(figsize=(12,6))
ax = sns.countplot(x="interest_level", hue="day_of_week",
                   hue_order=['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], data=df,
                   order=['low', 'medium', 'high'])
plt.xlabel('Day of Week')
plt.ylabel('Number of occurrences')

# Adding percents over bars
height = [p.get_height() for p in ax.patches]
ncol = int(len(height)/3)
total = [height[i] + height[i + ncol] + height[i + 2*ncol] for i in range(ncol)] * 3
for i, p in enumerate(ax.patches):    
    ax.text(p.get_x()+p.get_width()/2,
            height[i] + 50,
            '{:1.0%}'.format(height[i]/total[i]),
            ha="center") 

In [ ]:
# Iterest per weekend
fig = plt.figure(figsize=(12,6))
ax = sns.countplot(x="weekend", hue="interest_level", hue_order=['low', 'medium', 'high'],
                   data=df);
plt.xlabel('weekend');
plt.ylabel('Number of occurrences')

# Adding percents over bars
height = [p.get_height() for p in ax.patches]
ncol = int(len(height)/3)
total = [height[i] + height[i + ncol] + height[i + 2*ncol] for i in range(ncol)] * 3
for i, p in enumerate(ax.patches):    
    ax.text(p.get_x()+p.get_width()/2,
            height[i] + 50,
            '{:1.0%}'.format(height[i]/total[i]),
            ha="center") 

In [13]:
# price per bedroom
df.bedrooms[df.bedrooms == 0] = 1
df['price_per_bedroom'] = np.round(df.price / df.bedrooms, 0)

In [ ]:
# price outlier 확인
plt.scatter(range(df.shape[0]), np.sort(df.price.values))
plt.title("Price with outliers present",fontsize = 18)
plt.xlabel('Index', fontsize = 15)
plt.ylabel('Price', fontsize = 15)
plt.show()

In [ ]:
# Cut out bottom and top 1% of price data
price_low = np.percentile(df['price'], 1)
price_high = np.percentile(df['price'], 99)
    
    
# cut out the defined range above from the dataframe
df = df.drop(df[df['price'] < price_low].index)
df = df.drop(df[df['price'] > price_high].index)

In [ ]:
# price outlier 확인
plt.scatter(range(df.shape[0]), np.sort(df.price.values))
plt.title("Price with outliers present",fontsize = 18)
plt.xlabel('Index', fontsize = 15)
plt.ylabel('Price', fontsize = 15)
plt.show()

In [ ]:
# price histogram
def price_histogram():
    fig,ax = plt.subplots(1,2, figsize=(20,8))
    ax1,ax2 = ax.flatten()

    df.price.plot.hist(bins=20, color='Purple', edgecolor='black', ax=ax1);
    ax1.set_title('Price Histograms', fontsize=17)
    ax1.set_ylabel('')
    ax1.set_xlabel('')

    log_price = np.log(df.price)
    sns.distplot(log_price,  bins=30, color='tomato', kde=True, ax=ax2)
    ax2.set_title("Price Distribution Histogram", fontsize=17)
    ax2.set_ylabel("")
    ax2.set_xlabel('')
    plt.subplots_adjust(wspace=0.2);

price_histogram()

In [ ]:
# Log price 
df['log_price'] = df.price.map(np.log)

In [14]:
# create feature for number of photos, description length
df['num_of_photos'] = df.photos.map(len)
df.drop('photos', axis=1, inplace=True)
df['description_length'] = df.description.apply(lambda x: len(x.split(" ")))
df.drop('description', axis=1, inplace=True)
# bedroom = 0 이면 sutdio column 추가
df['studio'] = df.bedrooms.apply(lambda x: 1 if x==0 else 0)

In [ ]:
# Remove bathrooms outlier 
df.loc[df.bathrooms == 112, "bathrooms"] = 1

In [ ]:
# NY 벗어나는 위도/경도(lat; 40.6976684, long; -74.2605443) outlier 제거
long_low = -74.2
long_high = -73.7
lat_low = 40.5
lat_high = 41

#Removing out of bounds longitude
df = df.drop(df[df.longitude < long_low].index)
df = df.drop(df[df.longitude > long_high].index)

#Removing out of bounds latitude
df = df.drop(df[df.latitude < lat_low].index)
df = df.drop(df[df.latitude > lat_high].index)

In [ ]:
sns.pairplot(df[['longitude', 'latitude', 'interest_level']], hue='interest_level')
plt.ylabel('latitude')
plt.xlabel('longitude')
plt.show()

In [15]:
# feature 
df['features'].tolist()

# 하나의 리스트로 바꿔주고 모두 소문자로 변환
def flatten(vals):
    ret = []
    for i in vals:
        ret.extend([f.lower() for f in i])
    return ret

features = flatten(df['features'].tolist())

# 카운트 
count = dict()
for f in features:
    count[f] = count.get(f, 0) + 1
    
# 내림차순으로 정렬
sorted(count.items(), key=lambda x: x[1], reverse=True)

[('elevator', 26273),
 ('hardwood floors', 23558),
 ('cats allowed', 23540),
 ('dogs allowed', 22035),
 ('doorman', 20967),
 ('dishwasher', 20806),
 ('laundry in building', 18944),
 ('no fee', 18079),
 ('fitness center', 13257),
 ('laundry in unit', 9435),
 ('pre-war', 9149),
 ('roof deck', 6555),
 ('outdoor space', 5270),
 ('dining room', 5150),
 ('high speed internet', 4299),
 ('balcony', 3058),
 ('swimming pool', 2730),
 ('new construction', 2608),
 ('terrace', 2313),
 ('exclusive', 2167),
 ('loft', 2101),
 ('garden/patio', 1943),
 ('wheelchair access', 1358),
 ('prewar', 1349),
 ('common outdoor space', 1293),
 ('hardwood', 1058),
 ('fireplace', 919),
 ('simplex', 908),
 ('lowrise', 789),
 ('garage', 756),
 ('laundry room', 724),
 ('reduced fee', 699),
 ('furnished', 690),
 ('multi-level', 632),
 ('high ceilings', 613),
 ('private outdoor space', 534),
 ('publicoutdoor', 423),
 ('parking space', 418),
 ('roof-deck', 397),
 ('live in super', 366),
 ('renovated', 337),
 ('pool', 323)

In [ ]:
# 기존 feature 갯수와 count.key 갯수 비교
len(features), len(count.keys())

In [16]:
# 상위 10개 features value를 새로운 column으로 생성
def check(features, vals):
    for f in features:
        f = f.lower()
        if f in vals:
            return True
    return False

In [17]:
df['elevator'] = df.apply(lambda row: check(row['features'], ['elevator']), axis=1)

In [18]:
df['hardwood_floors'] = df.apply(lambda row: check(row['features'], ['hardwood floors', 'hardwood', 'hardwood floor', 'hardwood flooring' ]), axis=1)

In [19]:
pet_allowed = ['cats allowed', 'dogs allowed', 'pets allowed case by case', 'pets on approval', 'pet friendly', 'pets ok', 'pets', 'pets: cats/small dogs']
df['pet_allowed'] = df.apply(lambda row: check(row['features'], pet_allowed), axis=1)

In [20]:
doorman = ['doorman', 'full-time doorman', 'virtual doorman', 'ft doorman', '24/7 doorman', '24hr doorman', 'part-time doorman', '24/7 doorman concierge', '24-hour doorman', '24 hr doorman', '24 hour doorman', '24/7 full-time doorman concierge', ]
df['doorman'] = df.apply(lambda row: check(row['features'], doorman), axis=1)

In [21]:
df['dishwasher'] = df.apply(lambda row: check(row['features'], ['dishwasher', 'dish washer']), axis=1)

In [22]:
laundry = ['laundry in building', 'laundry in unit', 'on-site laundry', 'laundry room', 'laundry']
df['laundry'] = df.apply(lambda row: check(row['features'], laundry), axis=1)

In [23]:
df['no_fee'] = df.apply(lambda row: check(row['features'], ['no fee', 'reduced fee', 'low fee']), axis=1)

In [24]:
fitness_center = ['fitness center', 'gym/fitness', 'fitness room', 'state-of-the-art fitness center', 'fitness facility', 'fully-equipped club fitness center', 'state-of-the-art cardio and fitness club']
df['fitness_center'] = df.apply(lambda row: check(row['features'], fitness_center), axis=1)

In [25]:
df['pre_war'] = df.apply(lambda row: check(row['features'], ['pre-war', 'prewar', 'pre war']), axis=1)

In [26]:
roof_deck = ['roof deck', 'roof-deck', 'roofdeck', 'common roof deck', 'private roof deck', 'private roofdeck', 'roof deck with grills', 'roof deck with real grass', 'private roof-dec']
df['roof_deck'] = df.apply(lambda row: check(row['features'], roof_deck), axis=1)

In [27]:
df

,bathrooms,bedrooms,building_id,display_address,features,latitude,listing_id,longitude,manager_id,price,...,elevator,hardwood_floors,pet_allowed,doorman,dishwasher,laundry,no_fee,fitness_center,pre_war,roof_deck
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,Metropolitan Avenue,[],40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,3000,...,False,False,False,False,False,False,False,False,False,False
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,5465,...,True,False,True,True,False,False,False,True,False,False
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,2850,...,False,True,True,False,True,True,False,False,False,False
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,East 49th Street,"[Hardwood Floors, No Fee]",40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,3275,...,False,True,False,False,False,False,True,False,False,False
100013,1.0,4,0,West 143rd Street,[Pre-War],40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,3350,...,False,False,False,False,False,False,False,False,True,False
100014,2.0,4,38a913e46c94a7f46ddf19b756a9640c,West 18th Street,[],40.7429,6894514,-74.0028,b209e2c4384a64cc307c26759ee0c651,7995,...,False,False,False,False,False,False,False,False,False,False
100016,1.0,2,3ba49a93260ca5df92fde024cb4ca61f,West 107th Street,"[prewar, elevator, Dogs Allowed, Cats Allowed,...",40.8012,6930771,-73.9660,01287194f20de51872e81f660def4784,3600,...,True,True,True,False,False,False,True,False,True,False
100020,2.0,1,0372927bcb6a0949613ef5bf893bbac7,West 21st Street,"[Doorman, Elevator, Pre-War, Terrace, Laundry ...",40.7427,6867392,-73.9957,e6472c7237327dd3903b3d6f6a94515a,5645,...,True,True,False,True,True,True,False,False,True,False
100026,1.0,1,a7efbeb58190aa267b4a9121cd0c88c0,Hamilton Terrace,"[Cats Allowed, Dogs Allowed, Elevator, Laundry...",40.8234,6898799,-73.9457,c1a6598437b7db560cde66e5a297a53f,1725,...,True,False,True,False,False,True,False,False,False,False
100027,2.0,4,0,522 E 11th,"[Dishwasher, Hardwood Floors]",40.7278,6814332,-73.9808,23a01ea7717b38875f5b070282d1b9d2,5800,...,False,True,False,False,True,False,False,False,False,False


### RF로 feature Importances 알아보기

In [28]:
from sklearn import preprocessing
categorical = ['month', 'day_of_week', 'hour']
for f in categorical:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(df[f])
        df[f] = lbl.transform(df[f])

In [ ]:
# RF로 feature Importances 알아보기
from sklearn.ensemble import RandomForestClassifier
X = df[['bathrooms', 'bedrooms', 'price', 'price_per_bedroom', 'num_of_photos',\
        'description_length', 'weekend', 'studio', 'elevator', 'hardwood_floors',\
        'pet_allowed', 'doorman','dishwasher', 'laundry', 'no_fee', 'fitness_center',\
        'pre_war', 'roof_deck','month', 'day_of_week', 'hour']]
y = df['interest_level']

model1 = RandomForestClassifier(max_depth=10, n_estimators=100, random_state=0).fit(X, y)

In [ ]:
feature_names = ['bathrooms', 'bedrooms', 'price', 'price_per_bedroom', 'num_of_photos',\
        'description_length', 'weekend', 'studio', 'elevator', 'hardwood_floors',\
        'pet_allowed', 'doorman','dishwasher', 'laundry', 'no_fee', 'fitness_center',\
        'pre_war', 'roof_deck', 'month', 'day_of_week', 'hour']
importances = model1.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

### colums type 정리

In [ ]:
df_place = ['building_id', 'description', 'display_address ', 'latitude', 'listing_id', 'longitude'\
           'manager_id', 'street_address']
df_numeric = ['bathrooms', 'bedrooms', 'price', 'price_per_bedroom', 'log_price', 'num_of_photos'\
             'description_length']
df_categorical = ['month', 'day_of_week', 'hour']
df_bool = ['weekend', 'studio', 'elevator', 'hardwood_floors', 'pet_allowed', 'doorman',\
          'dishwasher', 'laundry', 'no_fee', 'fitness_center', 'pre_war', 'roof_deck']

In [ ]:
import pickle
with open("two-sigma-connect-rental-listing-inquiries/df_feature_engineering.pkl", "rb") as f:
    load_obj = pickle.load(f)
load_obj

### neighborhood column 추가  -> Clustering 사용?
- https://www.kaggle.com/luisblanche/price-compared-to-neighborhood-median

### building_id, manager_id 고민

In [ ]:
# df.building_id.value_counts()

In [ ]:
# df.manager_id.value_counts()